In [26]:
import unicycle_game as ug
import numpy as np
# system parameters
g = 9.81
l = 1
d = 1
mb = 70
mw = 5
Ib = 1/3 * mb * l**2
Iw = mw * (d/2)**2

# control parameters
kp = 1000
kv = 2 * kp**0.5
# initial conditions
y0 = np.array([0, 0])
dy0 = np.array([0, 0])

# compute d2y
def compute_dynamics(y, dy, f):
    M = np.array([[Ib + l**2 * mb, l * d * mb * np.cos(y[0])], [l * d * mb * np.cos(y[0]),   Iw + d**2 * (mb + mw)]])
    c = np.array([-l * g * mb * np.sin(y[0]), -dy[0]**2 * l * d * mb * np.sin(y[0])])
    d2y = np.linalg.inv(M).dot(f - c)
    return d2y

# controller for theta, note that there is only one actuator
def control_theta(y, dy, th_desired, dth_desired):
    th_error = th_desired - y[0]
    dth_error = dth_desired - dy[0]
    tau = kp * th_error + kv * dth_error
    f = np.array([tau, -tau]) # motor is connected between the two generalized coordinates
    return f

# update unicycle 
def update_unicycle(y, dy, th_desired, dth_desired, dt):
    f = control_theta(y, dy, th_desired, dth_desired)
    d2y = compute_dynamics(y, dy, f)
    # semi-implicit euler integration
    dy = dy + d2y * dt
    y = y + dy * dt
    return y, dy, d2y

In [20]:
try: # try to stop the game first in order to avoid creating redundant threads
    game.stop()
except NameError:
    pass
game = ug.Game(y0, dy0, update_unicycle, d, l)
game.start()
display(game.canvas)

MultiCanvas(height=300, width=600)

In [27]:
game.stop()